In [11]:
# import requests
# import json

# lat, lon = 35.9786, 126.7114

# url = (
#     f"https://climate-api.open-meteo.com/v1/climate?"
#     f"latitude={lat}&longitude={lon}"
#     f"&start_year=2020&end_year=2020"
#     f"&models=CMIP6"
#     f"&temperature_2m_mean=true"
#     f"&relative_humidity_2m_mean=true"
#     f"&cloudcover_mean=true"
#     f"&windspeed_10m_mean=true"
# )

# response = requests.get(url)

# if response.status_code == 200:
#     data = response.json()
#     print("✅ API 호출 성공!")
#     print(json.dumps(data, indent=2, ensure_ascii=False))
# else:
#     print(f"❌ API 호출 실패: {response.status_code}")
#     print(response.text)

❌ 실패 - 코드 401
{"cod":401, "message": "Invalid API key. Please see http://openweathermap.org/faq#error401 for more info."}


In [ ]:
# import requests

# API_KEY = "13a24ecc3a2785b9652b8816b02a8e55"
# lat = 35.9786
# lon = 126.7114
# start = 1609459200   # 2021-01-01 00:00:00 UTC
# end = 1609545600     # 2021-01-02 00:00:00 UTC

# url = f"https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={start}&end={end}&appid={API_KEY}"

# response = requests.get(url)

# if response.status_code == 200:
#     print("✅ 역사적 날씨 데이터 수신 성공!")
#     print(response.json())
# else:
#     print(f"❌ 실패 - 코드 {response.status_code}")
#     print(response.text)

In [15]:
pip install meteostat

Note: you may need to restart the kernel to use updated packages.


In [21]:
# from meteostat import Point, Daily
# from datetime import datetime

# # 위치: 군산
# location = Point(35.9786, 126.7114)

# # 조회 기간 (2023년 3월 1일 ~ 31일)
# start = datetime(2023, 3, 1)
# end = datetime(2023, 3, 31)

# data = Daily(location, start, end)
# data = data.fetch()

# print(data.head())

In [25]:
# from meteostat import Point, Daily
# from datetime import datetime
# import pandas as pd

# # 위치: 군산
# location = Point(35.9786, 126.7114)

# # 기간
# start = datetime(2023, 3, 1)
# end = datetime(2023, 3, 31)

# # 데이터 가져오기
# data = Daily(location, start, end).fetch()

# # ✨ 날짜 컬럼이 문자열일 경우 대비: 인덱스를 datetime으로 변환
# data.index = pd.to_datetime(data.index)

# # 확인
# print(data.head())
# # print(f"\n✅ 인덱스 타입 확인: {type(data.index)}")

            tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
time                                                                     
2023-03-01   6.2   0.4  10.1   0.4   NaN  342.0  15.8   NaN  1023.0   NaN
2023-03-02   2.7  -0.4   5.4   0.0   NaN  335.0  24.4   NaN  1028.4   NaN
2023-03-03   2.6  -4.0   8.0   0.0   NaN  167.0   9.3   NaN  1028.6   NaN
2023-03-04   4.6  -0.5  11.5   0.0   NaN  358.0   8.7   NaN  1028.6   NaN
2023-03-05   5.4  -0.7  14.0   0.0   NaN  357.0   5.9   NaN  1027.6   NaN


In [31]:
from meteostat import Point, Daily
from datetime import datetime, timedelta
import pandas as pd
import requests

# 도시명을 위도/경도로 변환하는 함수
def get_coords_from_city(city_name):
    try:
        url = f"https://nominatim.openstreetmap.org/search?q={city_name}&format=json&limit=1"
        headers = {"User-Agent": "weather-capstone-app"}
        response = requests.get(url, headers=headers).json()

        if response:
            lat = float(response[0]["lat"])
            lon = float(response[0]["lon"])
            return city_name, lat, lon
        else:
            raise ValueError("도시 이름을 찾을 수 없습니다.")

    except Exception as e:
        print("❌ 도시명 변환 실패:", e)
        new_city = input("🚨 유효한 도시명을 다시 입력해 주세요: ")
        return get_coords_from_city(new_city)

# IP로 위치 조회 실패 시 입력받는 함수
def get_location_from_ip_or_input():
    try:
        response = requests.get("https://ipapi.co/json/").json()
        city = response.get("city")
        lat = response.get("latitude")
        lon = response.get("longitude")

        if city is None or lat is None or lon is None:
            raise ValueError("위치 정보 누락")

        return city, lat, lon

    except Exception as e:
        print("❌ 위치 조회 실패:", e)
        user_city = input("🔎 자동 위치 조회에 실패했습니다. 도시명을 직접 입력해 주세요 (예: Seoul): ")
        return get_coords_from_city(user_city)

# 오늘 날짜 기준 ±15일
today = datetime.today()
start = today - timedelta(days=15)
end = today + timedelta(days=15)

# 위치 조회
city, lat, lon = get_location_from_ip_or_input()
print(f"📍 사용된 위치: {city} (위도 {lat}, 경도 {lon})")
print(f"📅 조회 기간: {start.date()} ~ {end.date()}")

# 데이터 수집
location = Point(lat, lon)
data = Daily(location, start, end).fetch()
data.index = pd.to_datetime(data.index)

# 평균값 계산
mean_values = data.mean(numeric_only=True)

# 결과 출력
print("\n📊 31일 평균 기상 데이터:")
print(mean_values.round(2))


❌ 위치 조회 실패: 위치 정보 누락


🔎 자동 위치 조회에 실패했습니다. 도시명을 직접 입력해 주세요 (예: Seoul):  군산


📍 사용된 위치: 군산 (위도 35.9993608, 경도 126.7602959)
📅 조회 기간: 2025-03-13 ~ 2025-04-12



📊 31일 평균 기상 데이터:
tavg      12.11
tmin       6.56
tmax      19.36
prcp       0.00
snow        NaN
wdir     186.62
wspd      21.95
wpgt        NaN
pres    1010.00
tsun        NaN
dtype: float64
